In [ ]:
import os
import torch
os.environ['HF_HOME'] = '/home/tl688/scratch/'
os.environ['HF_HUB_CACHE'] = '/home/tl688/scratch/'
os.environ['HF_TOKEN'] = ''

In [3]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info


In [2]:
import pandas as pd
image_test = pd.read_pickle("../../scratch_delete/adata_test_brain.pkl")
iamge_list_test = image_test[0] + image_test[1]

In [4]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info


model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "/home/tl688/scratch/saves/qwenbrain/full/sft/",
    torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json: 0.00B [00:00, ?B/s]

In [ ]:
generated_cell = []

with torch.cuda.amp.autocast():
    for idx in range(len(iamge_list_test)):
        messages = [
            {   "role": "system",
                "content": ""},
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "image": f"./brain_image/{idx}_brain.png",
                    },
                    {"type": "text", "text": 'Generate a list of 100 genes in order of descending expression from one spot shown in the histopathology image in brain tissue.\nCell sentence:'},       
                ],
            }
        ]
        # Preparation for inference
        text = processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        image_inputs, video_inputs = process_vision_info(messages)
        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        )
        inputs = inputs.to(model.device)

        # Inference: Generation of the output
        generated_ids = model.generate(**inputs, max_new_tokens=2048)
        generated_ids_trimmed = [
            out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        output_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )

        generated_cell.append(output_text[0])

/tmp/ipykernel_3309977/3575527514.py:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


In [ ]:
import pickle 
with open('./brain_image/generated_brain_ftqwen2.5_output.pkl', 'wb') as file:
    pickle.dump(generated_cell, file)